<h1 style="text-align: center;"> Assignment - 1 </h1>
<h4 style="text-align: center;">  Name: Shatansh Patnaik </h4>
<h4 style="text-align: center;"> Roll No: 20MA20067 </h4>

# Task -1:
Importing the Libraries needed for the preprocessing of the texts in the assignment. We'll be requiring spaCy for using NLP based functions for processing and modelling the text. We will be requiring the Regex library (re) for the pre-processing related task such as to remove all characters except for alphanumerics and whitespaces. We'll be requiring Pandas to load the dataset and perform the needed pre-processing

In [1]:
# Importing the essential libraries
import spacy
import re
import pandas as pd

In this step, we are displaying the data obtain from all the CSV files provided to us, we store the data obtained from docs.csv in a Dataframe named df, the data obtained from queries.csv in queries and the data from qdrel.csv as qdrel. 

In [2]:
# Importing the CSV files in the form of Dataframes
df = pd.read_csv("./Query_Doc/docs.csv")
queries = pd.read_csv("./Query_Doc/queries.csv")
qdrel = pd.read_csv("./Query_Doc/qdrel.csv")

print("\033[1m" + "Docs Data Frame: \n" + '\033[0m\n', df.head())
print("\033[1m" + "Queries Data Frame: \n" + '\033[0m\n', queries.head())
print("\033[1m" + "Relational Data Frame: \n" + '\033[0m\n', qdrel.head())

Docs Data Frame: 

    Unnamed: 0  doc_id                                           doc_text
0           0       1  What is the step by step guide to invest in sh...
1           1       2  What is the step by step guide to invest in sh...
2           2       3  What is the story of Kohinoor (Koh-i-Noor) Dia...
3           3       4  What would happen if the Indian government sto...
4           4       5  How can I increase the speed of my internet co...
Queries Data Frame: 

    Unnamed: 0  query_id                                         query_text
0           0      4584                How can ask questions using photos?
1           1      6588  What is Atal Pension Yojana? What are its bene...
2           2     10113      Where is starch digested? How is it digested?
3           3      7957      What is a conjecture? What are some examples?
4           4      5498  What can India do to support the people suffer...
Relational Data Frame: 

    Unnamed: 0  query_id  doc_id
0          

Now we realise that there is an unnecessary column named unnamed, we seek to remove that to filter out noise from the data set because it is **NOT** needed in the modelling process 

In [3]:
# Removal of the unnamed column from the dataframe
df = df.drop(df.columns[df.columns.str.contains('Unnamed', case=False)], axis=1)
queries = queries.drop(queries.columns[queries.columns.str.contains('Unnamed', case=False)], axis=1)
qdrel = qdrel.drop(qdrel.columns[qdrel.columns.str.contains('Unnamed', case=False)], axis=1)

print("\033[1m" + "Docs Data Frame: \n" + '\033[0m\n', df.head())
print("\033[1m" + "Queries Data Frame: \n" + '\033[0m\n', queries.head())
print("\033[1m" + "Relational Data Frame: \n" + '\033[0m\n', qdrel.head())

Docs Data Frame: 

    doc_id                                           doc_text
0       1  What is the step by step guide to invest in sh...
1       2  What is the step by step guide to invest in sh...
2       3  What is the story of Kohinoor (Koh-i-Noor) Dia...
3       4  What would happen if the Indian government sto...
4       5  How can I increase the speed of my internet co...
Queries Data Frame: 

    query_id                                         query_text
0      4584                How can ask questions using photos?
1      6588  What is Atal Pension Yojana? What are its bene...
2     10113      Where is starch digested? How is it digested?
3      7957      What is a conjecture? What are some examples?
4      5498  What can India do to support the people suffer...
Relational Data Frame: 

    query_id  doc_id
0       318     317
1       378     377
2       379     380
3       399    2606
4       399    2607


## a) Preprocessing of the docs and queries - removing the characters other than alphanumerics or whitespaces

Now the first sub task under task-1 expects us to remove all the characters other than alphanumerics and white spaces, we will be using regex for the same. The library used is re, which we had already imported. All we have to do is write a purify_docs function that takes in the data from each of the cell of the docs and queries dataframes as input and then apply the regex operation of filtering out the characters other than lower-case a-z, upper-case A-Z, digits from 0-9 and spaces.

In [4]:
# Removal of characters other than alphabets (both uppercase and lowercase included), digits and numbers
def purify_docs(data):
    purified_doc = re.sub(r'[^A-Za-z0-9]+|\s+', ' ', data)
    return purified_doc

df['pure'] = (df['doc_text']).apply(purify_docs)
queries['pure'] = (queries['query_text']).apply(purify_docs)

print("\033[1m" + "Docs Data Frame: \n" + '\033[0m\n', df.head(), "\n")
print("\033[1m" + "Queries Data Frame: \n" + '\033[0m\n', queries.head(), "\n")

Docs Data Frame: 

    doc_id                                           doc_text  \
0       1  What is the step by step guide to invest in sh...   
1       2  What is the step by step guide to invest in sh...   
2       3  What is the story of Kohinoor (Koh-i-Noor) Dia...   
3       4  What would happen if the Indian government sto...   
4       5  How can I increase the speed of my internet co...   

                                                pure  
0  What is the step by step guide to invest in sh...  
1  What is the step by step guide to invest in sh...  
2  What is the story of Kohinoor Koh i Noor Diamond   
3  What would happen if the Indian government sto...  
4  How can I increase the speed of my internet co...   

Queries Data Frame: 

    query_id                                         query_text  \
0      4584                How can ask questions using photos?   
1      6588  What is Atal Pension Yojana? What are its bene...   
2     10113      Where is starch digested?

## b) Now we need to correct the spellings in both queries and documents. 
For each query, which got corrected, we need to display the original and the corrected query on two spearate lines

In [5]:
# Correction of spellings in the dataset
from spellchecker import SpellChecker
 
spell = SpellChecker()
nlp = spacy.load("en_core_web_sm")

corrections = {}
def correctSpellings(sentence):
    doc = nlp(sentence)
    newList = []
    flag = 0
    for token in doc:
        if token.pos_ == 'PROPN' or token.ent_type_ != '':
            newList.append(token.text)
            continue
        else:
            misspelt = spell.unknown([token.text])
            if misspelt != set():
                newList.append(str(spell.correction(token.text)))
                flag = 1
            else:
                newList.append(token.text)
    if(flag):
        corrections[sentence] = ' '.join(newList)   
    return newList

def makeSentence(listOfWords):
    return ' '.join(listOfWords)

queries['correctList'] = queries['pure'].apply(correctSpellings)
queries['correctSentence'] = queries['correctList'].apply(makeSentence)

for key, value in corrections.items():
    print(f"\033[1mOriginal:\033[0m {key}\n\033[1mCorrection:\033[0m {value}\n")

Original: Kindly tell me whole process of admission at vits Vellore for biotech i m a bio student in 12 I don t have math there 
Correction: Kindly tell me whole process of admission at its Vellore for biotech i m a bio student in 12 I don t have math there

Original: How much equity should I get as CTO co founder 
Correction: How much equity should I get as CTO to founder

Original: What are the effects of demonitization of 500 and 1000 rupees notes on real estate sector 
Correction: What are the effects of demonetization of 500 and 1000 rupees notes on real estate sector

Original: What is the G len movement 
Correction: What is the G let movement

Original: How do you feel when someone upvotes your answer on Quora 
Correction: How do you feel when someone votes your answer on Quora

Original: Who are some famous people with low IQs 
Correction: Who are some famous people with low is

Original: Which microcontrollers are generally used in the robotics industry 
Correction: Which None

## c) Now we shall tokenize the words in the documents using spaCy library. 
Also as mentioned in the problem statement we need to remove all words that occur in less than 5 documents or more than 85% of the documents : this forms the vocabulary for the task. Lastly for each document and query, we create the create TF-IDF vectors for obtianing the cosine-similarity scores and further processing.

In [6]:
# First of all we need to load the NLP spaCy pipeline
nlp = spacy.load("en_core_web_sm")

# Next in order to tokenize the words into a list, we use the mentioned below function for dividing the words into tokens, for each cell of the dataframe a list is returned
def deriveTokens(sentence):
    doc = nlp(sentence)
    tokensList = []
    
    for token in doc:
        word = token.text
        tokensList.append(word)
        
    return tokensList


# Now we apply the above mentioned function to the dataframes
df['tokensList'] = df['pure'].apply(deriveTokens)
queries['tokensList'] = queries['pure'].apply(deriveTokens)

In [7]:
# Next we shall generate the vocabulary and for that purpose we first need a collection of all tokens used in the docs database
def performFiltrationOfWords(df):
    collectionOfAllWords = []

    for tokens in df['tokensList']:
        for eachToken in tokens:
            collectionOfAllWords.append(eachToken)

    collectionSet = set(collectionOfAllWords)
    mapCollections = {}

    for token in collectionSet:
        mapCollections[token] = collectionOfAllWords.count(token)

    purifiedWords = []
    
    for token in collectionSet:
        if mapCollections[token] >= 5 and mapCollections[token]<= len(df) * 0.85:
            purifiedWords.append(token)
            
    return purifiedWords

filteredWords = performFiltrationOfWords(df)
print("Length of Unique words list for Task-1: ", len(filteredWords))

# We generate a new column tokenList which has the words included in the given vocabulary and then we join those words to form sentences
df['tokensList'] = df['tokensList'].apply(lambda words: [word for word in words if word in filteredWords])
df['sentences'] = df['tokensList'].apply(lambda word: ' '.join(word))
queries['sentences'] = queries['tokensList'].apply(lambda word: ' '.join(word)) 

Length of Unique words list for Task-1:  2495


Next we apply the TFIDF function to the Docs and Queries data-frames post preprocessing, in order to create TFIDF vectors 

In [8]:
# Importing and Utilizing TFIDF Vectorizer for Vectorizing documents
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidfVectorsForDocs = tfidf.fit_transform(df['sentences'])
tfidfVectorsForQueries = tfidf.transform(queries['sentences'])

## d) Now for each of the queries, we shall perform the Cosine Pairwise Similarity
We will be using the sklearn library for the same to calculate the cosine pairwise similarity and along with that we will also be displaying the top 1, top 5 and top 10 most similar documents sorted according to Cosine Similarity scores for eahc query in the Queries dataframe

In [9]:
# Importing the cosing similarity library from sklearn
from sklearn.metrics.pairwise import cosine_similarity

generateCosineSimilarityMatrix = cosine_similarity(tfidfVectorsForQueries, tfidfVectorsForDocs)

# In this step we get the top k similar documents
def obtainSimilarDocs(n):
    return generateCosineSimilarityMatrix.argsort(axis=1)[:, -n:][:, ::-1]

# Function for printing similar docs
def printSimilarDocs(n, documents):
    print(f"\nTop {n} Similar Docs: ")
    for i, indices in enumerate(documents, start=1):
        docText = df.iloc[indices]['doc_text']
        docIndex = df.iloc[indices]['doc_id']
        print(f"Doc ID: {docIndex} : {docText}")
        
# Obtaining the similarity indices and printing them subsequently
topOneSimilarInds = obtainSimilarDocs(1)
topFiveSimilarInds = obtainSimilarDocs(5)
topTenSimilarInds = obtainSimilarDocs(10)

# Running a for loop in order to print all the values:
for i, rows in queries.iterrows():
    query = rows['query_text']
    print(f"\nGiven Query: {query}")
    
    printSimilarDocs(1, topOneSimilarInds[i])
    printSimilarDocs(5, topFiveSimilarInds[i])
    printSimilarDocs(10, topTenSimilarInds[i])


Given Query: How can ask questions using photos?

Top 1 Similar Docs: 
Doc ID: 1377 : What are some of the best photos?

Top 5 Similar Docs: 
Doc ID: 1377 : What are some of the best photos?
Doc ID: 1782 : What are the best interview questions to ask?
Doc ID: 9951 : Is there any way to automatically like Instagram photos with hashtags, using software?
Doc ID: 45 : What are the questions should not ask on Quora?
Doc ID: 4412 : Why do people have to ask Quora for questions?

Top 10 Similar Docs: 
Doc ID: 1377 : What are some of the best photos?
Doc ID: 1782 : What are the best interview questions to ask?
Doc ID: 9951 : Is there any way to automatically like Instagram photos with hashtags, using software?
Doc ID: 45 : What are the questions should not ask on Quora?
Doc ID: 4412 : Why do people have to ask Quora for questions?
Doc ID: 4583 : How do I ask questions with pictures on "Quora"?
Doc ID: 2603 : What are the best questions to ask a girl while chatting?
Doc ID: 9179 : How can I ma

## e) Calculations of Precision@K Scores

In [10]:
# Obtaining the P@K scores 
def getPScores(n):
    pAtOneSum = 0.0
    pAtFiveSum = 0.0
    pAtTenSum = 0.0
    if n==1:
        for i, rows in queries.iterrows():
            # We find the actual number of related docs for each query and then we calculate the number of intersections
            getRelations = qdrel[qdrel['query_id'] == rows['query_id']]['doc_id']
            generateSetOfRelations = set(getRelations)
            topOneDocs = topOneSimilarInds[i]
            dataAtOne = set()
            for j, indices in enumerate(topOneDocs, start=1):
                dataAtOne.add(df.iloc[indices]['doc_id'])
    
            pAtOne = len(generateSetOfRelations.intersection(dataAtOne)) / 1
            pAtOneSum += pAtOne
            queryLength = len(queries)
        return pAtOneSum/queryLength
    
    if n==5:
        for i, rows in queries.iterrows():
            getRelations = qdrel[qdrel['query_id'] == rows['query_id']]['doc_id']
            generateSetOfRelations = set(getRelations)
            topFiveDocs = topFiveSimilarInds[i]
            dataAtFive = set()
            for j, indices in enumerate(topFiveDocs, start=1):
                dataAtFive.add(df.iloc[indices]['doc_id'])
    
            pAtFive = len(generateSetOfRelations.intersection(dataAtFive)) / 5
            pAtFiveSum += pAtFive
            queryLength = len(queries)
        return pAtFiveSum/queryLength
    
    if n==10:
        for i, rows in queries.iterrows():
            getRelations = qdrel[qdrel['query_id'] == rows['query_id']]['doc_id']
            generateSetOfRelations = set(getRelations)
            topTenDocs = topTenSimilarInds[i]
            dataAtTen = set()
            for j, indices in enumerate(topTenDocs, start=1):
                dataAtTen.add(df.iloc[indices]['doc_id'])
    
            pAtTen = len(generateSetOfRelations.intersection(dataAtTen)) / 10
            pAtTenSum += pAtTen
            queryLength = len(queries)
        return pAtTenSum/queryLength
        
pAOne = getPScores(1)
pAFive = getPScores(5)
pATen = getPScores(10)

print(f"The Avg Precision@1 Score is as follows: {pAOne:.5f}")
print(f"The Avg Precision@5 Score is as follows: {pAFive:.5f}")
print(f"The Avg Precision@10 Score is as follows: {pATen:.5f}")

The Avg Precision@1 Score is as follows: 0.59000
The Avg Precision@5 Score is as follows: 0.19000
The Avg Precision@10 Score is as follows: 0.10100


## Task - 2a: Only Stemming :
SpaCy library has no builtin function for spacing, so we shallbe using the NLTK library for Stemming, essentially NLTK has three widely used Stemmers: 

- Porter Stemmer
- Snowball Stemmer
- Lancaster Stemmer

For the purpose of this model we will be using PorterStemmer, we will just creating an instance and then apply them to the words in the pre-processing phase. We have implemented all functions here, let us checkout the resultant Precision Scores at the end 

In [11]:
# Importing the NLTK library to perform stemming
from nltk.stem import PorterStemmer
st = PorterStemmer() 

# Usage of Stemming
def performStemmingAndTokenize(sentence):
    doc = nlp(sentence)
    tokensList = []
    
    for token in doc:
        word = st.stem(token.text)
        tokensList.append(word.lower())
        
    return tokensList

df['stemmedTokens'] = df['pure'].apply(performStemmingAndTokenize)
queries['stemmedTokens'] = queries['pure'].apply(performStemmingAndTokenize)

# Same processes as before to obtain the vocabulary
def performFiltrationOfStemmedWords(df):
    collectionOfAllWords = []

    for tokens in df['stemmedTokens']:
        for eachToken in tokens:
            collectionOfAllWords.append(eachToken)

    collectionSet = set(collectionOfAllWords)
    mapCollections = {}

    for token in collectionSet:
        mapCollections[token] = collectionOfAllWords.count(token)

    purifiedWords = []
    
    for token in collectionSet:
        if mapCollections[token] >= 5 and mapCollections[token]<= len(df) * 0.85:
            purifiedWords.append(token)
            
    return purifiedWords

filteredWords = performFiltrationOfStemmedWords(df)

df['stemmedTokens'] = df['stemmedTokens'].apply(lambda words: [word for word in words if word in filteredWords])
df['stemmedSentences'] = df['stemmedTokens'].apply(lambda word: ' '.join(word))
queries['stemmedSentences'] = queries['stemmedTokens'].apply(lambda word: ' '.join(word)) 

# Usage of TFIDF Vectorizer to vectorize the given dataset
tfidf = TfidfVectorizer()
tfidfVectorsForDocs = tfidf.fit_transform(df['stemmedSentences'])
tfidfVectorsForQueries = tfidf.transform(queries['stemmedSentences'])

# Calculation of the cosine Similarity Scores 
generateCosineSimilarityMatrix = cosine_similarity(tfidfVectorsForQueries, tfidfVectorsForDocs)
        
topOneSimilarInds = obtainSimilarDocs(1)
topFiveSimilarInds = obtainSimilarDocs(5)
topTenSimilarInds = obtainSimilarDocs(10)

# Calculation of Precision@K scores
pAOne = getPScores(1)
pAFive = getPScores(5)
pATen = getPScores(10)

print(f"The Avg Precision@1 Score is as follows: {pAOne:.5f}")
print(f"The Avg Precision@5 Score is as follows: {pAFive:.5f}")
print(f"The Avg Precision@10 Score is as follows: {pATen:.5f}")

The Avg Precision@1 Score is as follows: 0.69000
The Avg Precision@5 Score is as follows: 0.20000
The Avg Precision@10 Score is as follows: 0.10900


### Vocabulary size for Stemming (Number of Unique Words)

In [12]:
print(len(filteredWords))

2275


## Task - 2b: Performing Lemmatization on the Data
Now in this case, before the pre-processing step, we have the builtin function for a Lemmatizer in the standard spaCy NLP Pipeline, therefore we use it to provide only the base words for each of the words present in the data set

In [13]:
nlp = spacy.load("en_core_web_sm")
 
# Performance of Lemmatization 
def performLemmatizationAndTokenize(sentence):
    doc = nlp(sentence)
    tokensList = []
    
    for token in doc:
        tokensList.append(token.lemma_.lower())
        
    return tokensList

df['lemmatizedTokens'] = df['pure'].apply(performLemmatizationAndTokenize)
queries['lemmatizedTokens'] = queries['pure'].apply(performLemmatizationAndTokenize)

# Same processes as before to obtain the vocabulary
def performFiltrationOfLemmatizedWords(df):
    collectionOfAllWords = []

    for tokens in df['lemmatizedTokens']:
        for eachToken in tokens:
            collectionOfAllWords.append(eachToken)

    collectionSet = set(collectionOfAllWords)
    mapCollections = {}

    for token in collectionSet:
        mapCollections[token] = collectionOfAllWords.count(token)

    purifiedWords = []
    
    for token in collectionSet:
        if mapCollections[token] >= 5 and mapCollections[token]<= len(df) * 0.85:
            purifiedWords.append(token)
            
    return purifiedWords

filteredWords = performFiltrationOfLemmatizedWords(df)


df['lemmatizedTokens'] = df['lemmatizedTokens'].apply(lambda words: [word for word in words if word in filteredWords])
df['lemmatizedSentences'] = df['lemmatizedTokens'].apply(lambda word: ' '.join(word))
queries['lemmatizedSentences'] = queries['lemmatizedTokens'].apply(lambda word: ' '.join(word)) 

# Usage of TFIDF Vectorizer to vectorize the given dataset
tfidf = TfidfVectorizer()
tfidfVectorsForDocs = tfidf.fit_transform(df['lemmatizedSentences'])
tfidfVectorsForQueries = tfidf.transform(queries['lemmatizedSentences'])

# Calculation of the cosine Similarity Scores
generateCosineSimilarityMatrix = cosine_similarity(tfidfVectorsForQueries, tfidfVectorsForDocs)
        
topOneSimilarInds = obtainSimilarDocs(1)
topFiveSimilarInds = obtainSimilarDocs(5)
topTenSimilarInds = obtainSimilarDocs(10)

# Calculation of Precision@K scores
pAOne = getPScores(1)
pAFive = getPScores(5)
pATen = getPScores(10)

print(f"The Avg Precision@1 Score is as follows: {pAOne:.5f}")
print(f"The Avg Precision@5 Score is as follows: {pAFive:.5f}")
print(f"The Avg Precision@10 Score is as follows: {pATen:.5f}")

The Avg Precision@1 Score is as follows: 0.71000
The Avg Precision@5 Score is as follows: 0.19600
The Avg Precision@10 Score is as follows: 0.10800


### Vocabulary size for Lemmatization (Number of Unique Words)

In [14]:
print(len(filteredWords))

2234


## Task - 3: Apply Parts-Of-Speech Tagging and Named Entity Recognition and obtaining the results
In this case, we shall increase the frequency of the important words like the ones which are NOUNS and the ones which are names of entities like an organisation, person's name etc. We shall multiply in a factor of 2 for nouns and 4 for Named Entities

In [15]:
# Usage of TFIDF Vectorizer to vectorize the given dataset
tfidf = TfidfVectorizer()
tfidfVectorsForDocs = tfidf.fit_transform(df['lemmatizedSentences'])
tfidfVectorsForQueries = tfidf.transform(queries['lemmatizedSentences'])

tfidf_tokens = tfidf.get_feature_names_out() 
listOfWords = []

for word in tfidf_tokens:
    listOfWords.append(word)
    
listOfWords = list(nlp.pipe(listOfWords))

nounFactor = 2
nerFactor = 4

# Multiplication of a factor of 2 for proper nouns and 4 for named entities
for i in range(len(listOfWords)):
    word = listOfWords[i]
    for token in word:     
        if token.pos_ == "PROPN":  
            tfidfVectorsForDocs[:, i]*=nounFactor
            tfidfVectorsForQueries[:, i]*=nounFactor
        elif token.ent_type_ != '':
            tfidfVectorsForDocs[:, i]*=nerFactor
            tfidfVectorsForQueries[:, i]*=nerFactor

# Calculation of the cosine Similarity Scores            
generateCosineSimilarityMatrix = cosine_similarity(tfidfVectorsForQueries, tfidfVectorsForDocs)
        
topOneSimilarInds = obtainSimilarDocs(1)
topFiveSimilarInds = obtainSimilarDocs(5)
topTenSimilarInds = obtainSimilarDocs(10)

# Calculation of Precision@K scores
pAOne = getPScores(1)
pAFive = getPScores(5)
pATen = getPScores(10)

print(f"The Avg Precision@1 Score is as follows: {pAOne:.5f}")
print(f"The Avg Precision@5 Score is as follows: {pAFive:.5f}")
print(f"The Avg Precision@10 Score is as follows: {pATen:.5f}")

The Avg Precision@1 Score is as follows: 0.71000
The Avg Precision@5 Score is as follows: 0.19000
The Avg Precision@10 Score is as follows: 0.10200


## <ins>Task - 4: Changing the Parameters in order to get better results </ins> 

### <ins>**Changes**</ins> 

- **Rectifying the Vocabulary:** Instead of blindly flitering out the words which have frequency less than 5 and more than 85000, we will instead be removing all those words which are not a noun or aren't names of recognised entities and then successively, we'll apply lemmatization and then generate the words, also we note that we'll be using the POS Tagging as described above for words in their respective lower-case

> **Justification**: If we print the frequency maps or dictionaries containing the useful words, we observe that certain important words which are essential to the context of the sentence are lost because they have a frequency less than 5 in the whole dataset, this is the reason why we shall be using the above mentioned rectification

- **Usage of stopwords instead of manually eliminating higher frequency words:**

> **Justification:** Since we have removed the process of filtration, we still need to remove the function words that add nothing much significant to the context of the sentences. So this can easily be done by passing in the stop_words="english" parameter in the TFIDF Model

- **Conversion of all tokens to lowercase:**

> **Justification:** Since the same words (adding the same contextual meaning) in different cases create differetn types, so to prevent that and improve the matching, we convert everything to lowercase.

In [16]:
# First of all we need to load the NLP spaCy pipeline
nlp = spacy.load("en_core_web_sm")

# Next in order to tokenize the words into a list, we use the mentioned below function for dividing the words into tokens, for each cell of the dataframe a list is returned
def deriveNewTokens(sentence):
    doc = nlp(sentence)
    tokensList = []
    
    for token in doc:
        tokensList.append(token.lemma_.lower())
        
    return tokensList

# Now we apply the above mentioned function to the dataframes
df['tokensList'] = df['pure'].apply(deriveNewTokens)
df['newSentences'] = df['tokensList'].apply(lambda word: ' '.join(word)) 
queries['tokensList'] = queries['pure'].apply(deriveNewTokens)
queries['newSentences'] = queries['tokensList'].apply(lambda word: ' '.join(word)) 

# Usage of TFIDF Vectorizer to vectorize the given dataset
tfidf = TfidfVectorizer(analyzer='word',stop_words= 'english')
tfidfVectorsForDocs = tfidf.fit_transform(df['newSentences'])
tfidfVectorsForQueries = tfidf.transform(queries['newSentences'])

# Calculation of the cosine Similarity Scores  
generateCosineSimilarityMatrix = cosine_similarity(tfidfVectorsForQueries, tfidfVectorsForDocs)

topOneSimilarInds = obtainSimilarDocs(1)
topFiveSimilarInds = obtainSimilarDocs(5)
topTenSimilarInds = obtainSimilarDocs(10)

# Calculation of Precision@K scores
pAOne = getPScores(1)
pAFive = getPScores(5)
pATen = getPScores(10)

print(f"The Avg Precision@1 Score is as follows: {pAOne:.5f}")
print(f"The Avg Precision@5 Score is as follows: {pAFive:.5f}")
print(f"The Avg Precision@10 Score is as follows: {pATen:.5f}")

The Avg Precision@1 Score is as follows: 0.89000
The Avg Precision@5 Score is as follows: 0.22000
The Avg Precision@10 Score is as follows: 0.11700


In [17]:
# Running a for loop in order to print all the values:
for i, rows in queries.iterrows():
    query = rows['query_text']
    print(f"\nGiven Query: {query}")
    
    printSimilarDocs(1, topOneSimilarInds[i])
    printSimilarDocs(5, topFiveSimilarInds[i])
    printSimilarDocs(10, topTenSimilarInds[i])


Given Query: How can ask questions using photos?

Top 1 Similar Docs: 
Doc ID: 5828 : What is the best question that you have asked yourself?

Top 5 Similar Docs: 
Doc ID: 5828 : What is the best question that you have asked yourself?
Doc ID: 1377 : What are some of the best photos?
Doc ID: 1378 : What is a good photo?
Doc ID: 8413 : What does it mean when your crush asks you for your photo?
Doc ID: 3704 : How many questions have been asked on Quora?

Top 10 Similar Docs: 
Doc ID: 5828 : What is the best question that you have asked yourself?
Doc ID: 1377 : What are some of the best photos?
Doc ID: 1378 : What is a good photo?
Doc ID: 8413 : What does it mean when your crush asks you for your photo?
Doc ID: 3704 : How many questions have been asked on Quora?
Doc ID: 45 : What are the questions should not ask on Quora?
Doc ID: 5632 : What should be the first question to ask on Quora?
Doc ID: 3703 : How many questions have you asked on Quora?
Doc ID: 5827 : What is the most important qu